In [18]:
import requests
import json
import pprint
import pandas as pd
import pycurl
pp = pprint.PrettyPrinter()

In [2]:
# This is an example of one pull from the balldontlie.io API for a player searched by name
# You can see that lebron james was added as a search term at the end, and we can replicate this with other players
# The goal being to isolate the id's to use later for performance statistics
# Next we will be creating a for loop to get all of the players from our salaries list

In [21]:
# As you can see in this example, any capitalization does not matter for the player search
james = requests.get("https://www.balldontlie.io/api/v1/players/?search=LEbron jaMes")
james.json()

{'data': [{'id': 237,
   'first_name': 'LeBron',
   'height_feet': 6,
   'height_inches': 8,
   'last_name': 'James',
   'position': 'F',
   'team': {'id': 14,
    'abbreviation': 'LAL',
    'city': 'Los Angeles',
    'conference': 'West',
    'division': 'Pacific',
    'full_name': 'Los Angeles Lakers',
    'name': 'Lakers'},
   'weight_pounds': 250}],
 'meta': {'total_pages': 1,
  'current_page': 1,
  'next_page': None,
  'per_page': 25,
  'total_count': 1}}

In [4]:
# We take the file from BDLio and convert it to a dictionary so we car parse the ID
james_dict = json.loads(james.text)
print(james_dict)
type(james_dict)

{'data': [{'id': 237, 'first_name': 'LeBron', 'height_feet': 6, 'height_inches': 8, 'last_name': 'James', 'position': 'F', 'team': {'id': 14, 'abbreviation': 'LAL', 'city': 'Los Angeles', 'conference': 'West', 'division': 'Pacific', 'full_name': 'Los Angeles Lakers', 'name': 'Lakers'}, 'weight_pounds': 250}], 'meta': {'total_pages': 1, 'current_page': 1, 'next_page': None, 'per_page': 25, 'total_count': 1}}


dict

In [16]:
james_id = james_dict['data'][0]["id"]

237

In [20]:
%store -r player_names_list
player_names_list

['Stephen Curry',
 'John Wall',
 'Russell Westbrook',
 'James Harden',
 'Damian Lillard',
 'LeBron James',
 'Kevin Durant',
 'Giannis Antetokounmpo',
 'Paul George',
 'Kawhi Leonard',
 'Klay Thompson',
 'Jimmy Butler',
 'Tobias Harris',
 'Khris Middleton',
 'Anthony Davis',
 'Rudy Gobert',
 'Kyrie Irving',
 'Bradley Beal',
 'Kristaps Porziņģis',
 'Devin Booker']